# Data wrangling exercise

This notebook is available at https://github.com/alexyarosh/data_exercise/exercise.ipynb

In [1]:
import pandas as pd
import numpy as np
import json
import re

# read the jsonl

filename = 'ida_wrangling_exercise_data.2017-02-13.jsonl'
file  = open(filename)

records = []
for line in file:
    rec = json.loads(line)
    records.append(rec)


def flatten_dict(dictionary, remove=False):
    
    # flattens a nested dictionary
    # returns a new dictionary whose nested keys are concatenated by '.', 
    # the values of the top level keys are removed if 'remove' parameter is set to True, otherwise are set to NaN
    # example:
    # if nested_dict = {'pet': {'type' : 'cat', 'breed' : 'siamese'}}
    # flatten_dict(nested_dict, remove=False) returns
    # {'pet.type': 'cat', 'pet.breed':siamese, 'pet':NaN} 
    
    new_dict = dictionary.copy()
    for (key,value) in dictionary.items():
        if type(value) == dict:
            for (nested_key,nested_value) in value.items():
                new_key = key + '.' + nested_key
                new_dict[new_key] = nested_value
                if remove:
                    del new_dict[key]
                else:
                    new_dict[key] = np.nan                 
    return new_dict 

flattened_records = [flatten_dict(d) for d in records]
data = pd.DataFrame(flattened_records)   

## List of fields

In [2]:
# Print out the alphabetized list of fields
print('Alphabetized list of field names:\n')
print(sorted(data.columns.get_values().tolist()))

Alphabetized list of field names:

['address', 'address.city', 'address.state', 'address.street', 'address.zip', 'dob', 'email', 'id', 'name', 'name.firstname', 'name.lastname', 'name.middlename', 'phone', 'record_date', 'ssn']


## Summary statistics

In [3]:
# Count non-null values, divide by the total number of records, and multiply by 100 to get the percentage
print('Percentage of records containing the field:\n')
print(data.count()/data.shape[0]*100)

Percentage of records containing the field:

address             50.128667
address.city        40.822000
address.state       40.822000
address.street      40.822000
address.zip         40.822000
dob                 95.916000
email               87.253333
id                 100.000000
name                28.705333
name.firstname      70.002667
name.lastname       70.002667
name.middlename     29.112667
phone               93.507333
record_date        100.000000
ssn                 94.962667
dtype: float64


In [4]:
most_common = pd.DataFrame([])
for c in data.columns:
    most_common[c] = data[c].value_counts().index[:5]

print('Five most common values in each field:')

most_common.head()

Five most common values in each field:


,address,address.city,address.state,address.street,address.zip,dob,email,id,name,name.firstname,name.lastname,name.middlename,phone,record_date,ssn
0,"4951 Mallory Way Suite 905\nSouth Hollyton, SD...",New Michael,NC,107 Andrea Roads,99528,1950-11-13,zsmith@gmail.com,d40bf1d822e643e9838fc3946c32d9ca,David Smith,Michael,Smith,Michael,1-298-341-1025,2000-05-08T12:20:50,xxx-xx-6568
1,"7833 Flores Skyway\nEast Connie, CO 52630",Lake Michael,DC,6130 Mark Drive,26231,1963-01-25,kwilliams@yahoo.com,44d69658bb9d465a817bf8756638d2ce,John Smith,David,Johnson,David,+68(9)8268242444,2015-09-06T14:20:22,xxx-xx-2156
2,"0947 Jane Walk Suite 563\nWest Tamara, VA 0507...",East Michael,MI,1073 Harper Courts,50032,1964-06-11,qsmith@gmail.com,a56f207406d248189cdcdf653ae1f064,Michael Smith,James,Williams,Jennifer,(655)341-3810x5812,2005-10-11T21:17:11,xxx-xx-2027
3,"451 Wagner Trafficway\nNew Whitneyport, VI 65026",Port Michael,MD,02127 Ricardo Dale,98018,1963-11-30,asmith@hotmail.com,007b6225541444e089952ecf3a657471,Michael Johnson,Jennifer,Brown,James,(694)209-2148x2933,2013-11-22T18:45:19,xxx-xx-2798
4,"97441 Miller Junction Apt. 356\nLeemouth, VA 4...",North Michael,OH,002 Jones Drive,17086,1968-01-18,ejohnson@gmail.com,8d7d1c6aaf664ccf837c75b4af5c5e97,Joseph Smith,John,Jones,John,071-269-6432x55162,2003-05-23T16:21:08,xxx-xx-2963


## Names

First, we'll parse the `name` into first and last name. To do so, we first split the name string on whitespace, remove any prefices/suffices (e.g. 'Mrs.', 'PhD', 'Jr'), from the list, and assign the first remaining string to `firstname` and second remaining string to `lastname`



In [5]:
def parse_name(name):
    # takes 'Ms. Firstname Lastname PhD' and  returns [Firstname, Lastname]
    if pd.isnull(name):
        return [np.nan,np.nan]
    strings = name.split(" ")
    to_remove = ["Mrs.", "Ms.", "Miss", "Mr.", "Dr.", "MD", "PhD", "DDS", "DVM","Jr.", "II", "III", "IV", "V"]
    if strings[0] in to_remove:
           strings = strings[1:]
           
    if strings[-1] in to_remove:
           strings = strings[:-1]
           
    if len(strings) == 2:
        firstname = strings[0]
        lastname = strings[1]
    else:
        raise ValueError('Cannot process the name '+name)  
    
    return [firstname, lastname]

Now fill `name.firstname`, `name.lastname` from `name`: 

In [6]:
#this somehow turned out to be the hardest part of the exercise...

ser = data.loc[pd.notnull(data['name']), 'name'].apply(parse_name) #this returns a series of lists

#make the series of lists into a dataframe and change the column names to match data 
df = pd.DataFrame.from_items(zip(ser.index, ser.values)).T
df.columns = ['name.firstname', 'name.lastname']

#assign the new dataframe to the slice
data.loc[pd.notnull(data['name']), ('name.firstname', 'name.lastname')] = df

#check that the data looks good
data.head()

,address,address.city,address.state,address.street,address.zip,dob,email,id,name,name.firstname,name.lastname,name.middlename,phone,record_date,ssn
0,NaN,Hoodburgh,RI,86314 David Pass Apt. 211,83973,1971-06-30,opark@hotmail.com,01d68a4c598a45559c06f4df0b3d82cb,NaN,Cynthia,Dawson,Claire,624-869-4610,2006-07-08T09:02:13,xxx-xx-2412
1,"20722 Coleman Villages\nEast Rose, SC 71064-5894",NaN,NaN,NaN,NaN,1965-09-09,sperez@armstrong.com,876ff718291d4397bb1e0477ceee6ad9,NaN,Tamara,Myers,NaN,1-594-462-7759,2009-03-28T20:22:57,xxx-xx-8025
2,NaN,New Julie,UT,6676 Young Square,73125,1993-04-12,uortiz@gmail.com,81753097bf7e4e2085982f422bdb9cda,NaN,Jamie,Alexander,NaN,472.218.5065x389,2016-08-30T20:31:39,xxx-xx-0568
3,"0932 Gomez Drives\nLeefort, MD 46879-3166",NaN,NaN,NaN,NaN,1977-04-14,palmerdiane@yahoo.com,2c2f7154b80f40ca80d08c5adc54ea45,NaN,Angela,Garcia,Alexis,1-663-109-4460x1080,2001-02-15T18:50:35,xxx-xx-9825
4,NaN,East Sharonstad,ME,158 Smith Vista,42483,1970-03-19,nancymaxwell@gmail.com,4f5263f339694d068e17ee7fdbb852b8,NaN,Jennifer,Rodriguez,NaN,233-423-3823,2014-06-21T14:36:01,xxx-xx-9104


Now we can find the number of unique first names by just counting the number of unique entires in the `name.firstname` field

In [7]:
print("Number of unique first names: "+ str(pd.Series(data['name.firstname']).nunique()))

Number of unique first names: 690


## Street names

We'll start with a function to parse an address into a street address, city, state, and zipcode.

Examining the data, we see that the street address is separated from the rest by the newline characted `\n`, i.e. the addresses are of the form 

`'1234 Street Name Apt. 5\nCity, ST 09876'`

So the first step is to split the address on the occurence of `'\n'`. The first string contains the street address, and the second string consists of city name, followed by a comma and a whitespace, followed by a two-letter state abbreviation, whitespace, and the zipcode. So to extract the necessary information, it's sufficient to split the string on the occurence of ', '. On one side we'll get the city. On the other side, first two characters will give the state, and characters number 4 (3rd character is the space) and further will give the zipcode.


In [8]:
def parse_address(address):
    # takes an address of the form '1234 Street Name Apt. 5\nCity, ST 09876' 
    #and returns [1234 Street Name Apt. 5, City, ST, 09876]
    if pd.isnull(address):
        return [np.nan,np.nan,np.nan,np.nan]
    strings = address.split("\n")
    if ('APO ' in strings[1]) or ('FPO ' in strings[1]) or ('DPO ' in strings[1]): #military
        return [np.nan,np.nan,np.nan,np.nan]
    temp = strings[1].split(', ')
    city = temp[0]
    state = temp[1][0:2]
    zipcode = temp[1][3:]
    street_address = strings[0]
    return [street_address,city, state, zipcode] 


Now we fill  `address.street`,  `address.city`,  `address.state`,  `address.zip` based on parsing of `address`

In [9]:
#this somehow turned out to be the hardest part of the exercise...

ser = data.loc[pd.notnull(data['address']), 'address'].apply(parse_address) #this returns a series of lists

#make the series of lists into a dataframe and change the column names to match data 
df1 = pd.DataFrame.from_items(zip(ser.index, ser.values)).T
df1.columns = ['address.street', 'address.city', 'address.state', 'address.zip']

#assign the new dataframe to the slice
data.loc[pd.notnull(data['address']), ('address.street', 'address.city', 'address.state', 'address.zip')] \
= df1


Next, we look at a street address. A street address (`address.street`) will be split ino into 3 distinct fields containing building number, street name, and apartment. It is convenient to do so if we're interested in street names specifically.

Quick look at the data shows that the street addresses come in one of the three formats:
- `'1234 Street Name'`
- `'1234 Street Name Apt. 5'`
- `'1234 Street Name Suite 678'`

where the building number, street name, apartment/suite number can have any length.

To parse this into different field, we first split the string on the first occurence of the whitespace. Note that it has to be the first occurence becasue street name can contain spaces as well (in particular, we can't just use a 'splitting into words' function). 

The first out of the two resulting strings will give us the building number. Then we split the remaining string on the occurence of ' Apt.' and ' Suite ' (notice the leading spaces and the period/space at the end -- we don't want to split a street whose name just contains 'Apt'). Then the firt string resulting from this second splitting is the street name, and the last string -- apartment of suite number.

In [10]:
def parse_street_address(street_addr):
    # takes an address of the form '1234 Street Name' or '1234 Street Name Apt. 5' or '1234 Street Name Suite 678'
    #and returns a tuple or the form (1234, Street Name, NaN), (1234, Street Name, Apt. 5) or (1234, Street Name, Suite 678)
    if pd.isnull(street_addr):
        return np.nan, np.nan, np.nan    
    s1 = street_addr.split(' ', 1)
    house = s1[0]
    apt = s1[1].find(' Apt.')
    suite = s1[1].find(' Suite')
    if (apt == -1) and (suite == -1):  # no apartment number or suite number  
        street = s1[1]
        additional = np.nan
    elif (apt != -1) and (suite == -1): # contains an apartment number
        street = s1[1][0:apt]
        additional = s1[1][apt+1:]
    elif (apt == -1) and (suite != -1): # contains a suite number
        street = s1[1][0:suite]
        additional = s1[1][suite+1:]
    else:                               #???
        raise ValueError('Address contains both apartment number and suite number: '+street_addr)          
    
    return house, street, additional

And again, we record the results in the dataframe

In [11]:
data['address.building'],data['address.street_name'],data['address.apt'] = zip(*data['address.street'].apply(parse_street_address))

Now finding unique street names is easy -- just count the number of unique records in the `address.street_name` column

In [12]:
print("Number of unique street names: "+ str(pd.Series(data['address.street_name']).nunique()))

Number of unique street names: 79999


## Phone numbers

The phone numbers in the data are messed up. First, numbers like +44(9)3987289182 are not US phone numbers. The '+' prefix is used in international phone number standards to indicate that what follows it is a country code. A stringin this format represents a US number only if it starts with +1. On the other hand, looking. for example, at data[37], we get a number that can't be a US number because it doesn't start with +1, but also can't be an international number, because there are no country codes starting at 0.

Next, the 11-digit codes with leading zero also don't follow any US format. In countries that aren't the US, leading 0 is used for long-distance calling, but in the US 1 is used for the same purpose.

To get around these issues, we will assign a flag to each record, showing whether the phone number in a US phone number or not. We will also create a separate field to store extentisions, as follows:
1. if the number starts with + but isn't of the form +1(##)#########, we flag it as a non-US phone number and do nothing else
2. if the number start with '+1', flas is as a US phone number, and remove '+1'
3. if the number starts with '1-', remove the '1-'
4. split the string on x, and store the second part separately as extention
5. remove all occurences of '-', '.', and '('')'
6. flag the record as a US number if the resulting string has 10 characters, and as not a US number otherwise


In [13]:
def format_us_phone(phone):
    # phone number format
    # determines in the number provided is a valid US phone number, and 
    if pd.isnull(phone):
        return np.nan, np.nan, False
    if phone[0] == '+':
        if (phone[1:3] == '1('): #US country code
            phone = phone[2:] #remove +1
        else:    
            return phone, np.nan, False
        
    if phone[0:2] == '1-':
        phone = phone[2:]          

    phone_and_ext = phone.split('x') 
    phone_number = phone_and_ext[0]
    if len(phone_and_ext) > 1:
        ext = phone_and_ext[1]
    else:
        ext = np.nan
    phone_number = re.sub(r'\.?-?\)?\(?','', phone_number)
    
    if len(phone_number) != 10: #not a US phone number
        return phone, np.nan, False #return original
    
    return phone_number, ext, True

Create new fields in the dataframe, and make sure it looks alright:

In [14]:
data['phone.number'], data['phone.ext'], data['phone.isUS'] = zip(*data['phone'].apply(format_us_phone))
data.head()

,address,address.city,address.state,address.street,address.zip,dob,email,id,name,name.firstname,...,name.middlename,phone,record_date,ssn,address.building,address.street_name,address.apt,phone.number,phone.ext,phone.isUS
0,NaN,Hoodburgh,RI,86314 David Pass Apt. 211,83973,1971-06-30,opark@hotmail.com,01d68a4c598a45559c06f4df0b3d82cb,NaN,Cynthia,...,Claire,624-869-4610,2006-07-08T09:02:13,xxx-xx-2412,86314,David Pass,Apt. 211,6248694610,NaN,True
1,"20722 Coleman Villages\nEast Rose, SC 71064-5894",East Rose,SC,20722 Coleman Villages,71064-5894,1965-09-09,sperez@armstrong.com,876ff718291d4397bb1e0477ceee6ad9,NaN,Tamara,...,NaN,1-594-462-7759,2009-03-28T20:22:57,xxx-xx-8025,20722,Coleman Villages,NaN,5944627759,NaN,True
2,NaN,New Julie,UT,6676 Young Square,73125,1993-04-12,uortiz@gmail.com,81753097bf7e4e2085982f422bdb9cda,NaN,Jamie,...,NaN,472.218.5065x389,2016-08-30T20:31:39,xxx-xx-0568,6676,Young Square,NaN,4722185065,389,True
3,"0932 Gomez Drives\nLeefort, MD 46879-3166",Leefort,MD,0932 Gomez Drives,46879-3166,1977-04-14,palmerdiane@yahoo.com,2c2f7154b80f40ca80d08c5adc54ea45,NaN,Angela,...,Alexis,1-663-109-4460x1080,2001-02-15T18:50:35,xxx-xx-9825,0932,Gomez Drives,NaN,6631094460,1080,True
4,NaN,East Sharonstad,ME,158 Smith Vista,42483,1970-03-19,nancymaxwell@gmail.com,4f5263f339694d068e17ee7fdbb852b8,NaN,Jennifer,...,NaN,233-423-3823,2014-06-21T14:36:01,xxx-xx-9104,158,Smith Vista,NaN,2334233823,NaN,True


Finally, find the 5 most common area codes for US numbers

In [15]:
print('5 most common area codes:')
print(list(pd.Series(data[data['phone.isUS']]['phone.number'].str[0:3]).value_counts()[:5].index))

5 most common area codes:
['947', '488', '913', '092', '740']


## PS

While this submission is coming relatively late, I would ask you to consider the fact that I was at a [conference](http://www.ams.org/meetings/sectional/2250_program.html) during the weekend and therefore did not have as much time available to work on it (but I understand that you have a huge number of applicant, so extending this courtesy might not be possible).

In any case, thank you for a fun exercise, and I hope to hear from you soon!